# Analisi dei dati raccolti

## Setup

In [5]:
import pandas as pd

In [6]:
def print_data(series: pd.Series) -> None:
    print(series.value_counts())
    print()
    print(f"Massimo: {series.max()}\nMinimo: {series.min()}\nMedia: {series.mean()}\nVarianza: {series.var()}")

## Asconhashbi32v12

In [7]:
path = "../asconhashbi32v12/data/{}.csv"

### Bi32_armv7m

In [8]:
with open(path.format("bi32_armv7m"), "r") as f:
    ahbi32_bi32 = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [9]:
# Generazione hash
data = ahbi32_bi32["0B"]

print_data(data)

0B
66    800
67    200
Name: count, dtype: int64

Massimo: 67
Minimo: 66
Media: 66.2
Varianza: 0.1601601601601601


#### PT 8 byte

In [10]:
# Generazione hash
data = ahbi32_bi32["8B"]

print_data(data)

8B
80    997
82      2
81      1
Name: count, dtype: int64

Massimo: 82
Minimo: 80
Media: 80.005
Varianza: 0.008983983983983984


#### PT 16 byte

In [11]:
# Generazione hash
data = ahbi32_bi32["16B"]

print_data(data)

16B
95    997
96      2
94      1
Name: count, dtype: int64

Massimo: 96
Minimo: 94
Media: 95.001
Varianza: 0.0030020020020020007


#### PT 32 byte

In [12]:
# Generazione hash
data = ahbi32_bi32["32B"]

print_data(data)

32B
123    799
124    200
125      1
Name: count, dtype: int64

Massimo: 125
Minimo: 123
Media: 123.202
Varianza: 0.16335935935935933


#### PT 64 byte

In [13]:
# Generazione hash
data = ahbi32_bi32["64B"]

print_data(data)

64B
180    798
181    201
182      1
Name: count, dtype: int64

Massimo: 182
Minimo: 180
Media: 180.203
Varianza: 0.1639549549549549


#### PT 128 byte

In [14]:
# Generazione hash
data = ahbi32_bi32["128B"]

print_data(data)

128B
294    797
295    201
296      2
Name: count, dtype: int64

Massimo: 296
Minimo: 294
Media: 294.205
Varianza: 0.16714214214214215


#### PT 256 byte

In [15]:
# Generazione hash
data = ahbi32_bi32["256B"]

print_data(data)

256B
524    400
523    399
522    199
525      2
Name: count, dtype: int64

Massimo: 525
Minimo: 522
Media: 523.205
Varianza: 0.5655405405405405


#### PT 512 byte

In [16]:
# Generazione hash
data = ahbi32_bi32["512B"]

print_data(data)

512B
981    997
982      3
Name: count, dtype: int64

Massimo: 982
Minimo: 981
Media: 981.003
Varianza: 0.0029939939939939942


#### PT 1024 byte

In [17]:
# Generazione hash
data = ahbi32_bi32["1024B"]

print_data(data)

1024B
1895    997
1897      3
Name: count, dtype: int64

Massimo: 1897
Minimo: 1895
Media: 1895.006
Varianza: 0.011975975975975977


### Ref

In [18]:
with open(path.format("ref"), "r") as f:
    ahbi32_ref = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [19]:
# Generazione hash
data = ahbi32_ref["0B"]

print_data(data)

0B
640    510
639    362
638     86
641     39
637      3
Name: count, dtype: int64

Massimo: 641
Minimo: 637
Media: 639.496
Varianza: 0.5185025025025026


#### PT 8 byte

In [20]:
# Generazione hash
data = ahbi32_ref["8B"]

print_data(data)

8B
768    673
767    197
766    118
769     12
Name: count, dtype: int64

Massimo: 769
Minimo: 766
Media: 767.579
Varianza: 0.5042632632632632


#### PT 16 byte

In [21]:
# Generazione hash
data = ahbi32_ref["16B"]

print_data(data)

16B
897    443
896    432
895    117
898      8
Name: count, dtype: int64

Massimo: 898
Minimo: 895
Media: 896.342
Varianza: 0.47551151151151155


#### PT 32 byte

In [22]:
# Generazione hash
data = ahbi32_ref["32B"]

print_data(data)

32B
1155    744
1156    212
1157     44
Name: count, dtype: int64

Massimo: 1157
Minimo: 1155
Media: 1155.3
Varianza: 0.2982982982982982


#### PT 64 byte

In [23]:
# Generazione hash
data = ahbi32_ref["64B"]

print_data(data)

64B
1671    638
1670    283
1672     79
Name: count, dtype: int64

Massimo: 1672
Minimo: 1670
Media: 1670.796
Varianza: 0.3207047047047046


#### PT 128 byte

In [24]:
# Generazione hash
data = ahbi32_ref["128B"]

print_data(data)

128B
2704    633
2703    246
2702    121
Name: count, dtype: int64

Massimo: 2704
Minimo: 2702
Media: 2703.512
Varianza: 0.49234834834834834


#### PT 256 byte

In [25]:
# Generazione hash
data = ahbi32_ref["256B"]

print_data(data)

256B
4768    554
4767    235
4769    205
4770      6
Name: count, dtype: int64

Massimo: 4770
Minimo: 4767
Media: 4767.982
Varianza: 0.46414014014014005


#### PT 512 byte

In [26]:
# Generazione hash
data = ahbi32_ref["512B"]

print_data(data)

512B
8898    873
8897    118
8899      8
8900      1
Name: count, dtype: int64

Massimo: 8900
Minimo: 8897
Media: 8897.892
Varianza: 0.11845445445445445


#### PT 1024 byte

In [27]:
# Generazione hash
data = ahbi32_ref["1024B"]

print_data(data)

1024B
17158    786
17159    208
17157      6
Name: count, dtype: int64

Massimo: 17159
Minimo: 17157
Media: 17158.202
Varianza: 0.17336936936936934
